In [ ]:
import pandas as pd
import matplotlib as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
import itertools
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
#Extraindo os dados
data = pd.read_csv('Amazon Sale Report.csv',sep=";")

#Explorando os dados
# print(data.head())
# print(data.info())
# print(data.describe())


In [ ]:
#Limpando os dados e Filtrando para analise
data.dropna(inplace=True)

data['Date'] = pd.to_datetime(data['Date'])
data['Month'] = data['Date'].dt.month
data['Year'] = data['Date'].dt.year

le = LabelEncoder()
data['Fulfilment'] = le.fit_transform(data['Fulfilment'])
data['Category'] = le.fit_transform(data['Category'])

In [ ]:
#Selecionando as colunas para previsão
X = data[['Fulfilment', 'Category', 'Month']]
y = data['Amount']

#Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#Escolehndo o modelo Random Forest
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

#Valores do modelo
rf_predictions = rf.predict(X_test)
print("Random Forest MAE:", mean_absolute_error(y_test, rf_predictions))
print("Random Forest MSE:", mean_squared_error(y_test, rf_predictions))
print("Random Forest R2:", r2_score(y_test, rf_predictions))


In [ ]:
#Prevendo os valores futuros
fulfilment_unique = data['Fulfilment'].unique()
category_unique = data['Category'].unique()
month_unique = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
#Juntando as novas colunas
combinacao = list(itertools.product(fulfilment_unique, category_unique, month_unique))
vendas_futuras = pd.DataFrame(combinacao, columns=['Fulfilment', 'Category', 'Month'])
#Adicionando a coluna de previsão de vendas
vendas_futuras_scaled = scaler.transform(vendas_futuras)
vendas_futuras['predicted_sales'] = rf.predict(vendas_futuras_scaled)
vendas_futuras.info()



In [ ]:
#Visualizando as colunas com mais vendas
vendas_futuras.groupby('Category')['predicted_sales'].sum()

In [ ]:
#Encontrando as categorias em relação ao seu numero no labelEncoder
le.classes_

In [ ]:
# Somando os valores de cada categoria
category_sales = vendas_futuras.groupby('Category')['predicted_sales'].sum().sort_values(ascending=False)

# Indexando as 5 melhores categorias
index_cat = category_sales.head(5).index

# Filtrando para obter os dados das 5 categorias
filt_cat = vendas_futuras[vendas_futuras['Category'].isin(index_cat)]

# Agrupando os dados das 5 categorias nas duas colunas
cat_top5 = filt_cat.groupby('Category')['predicted_sales'].sum()

# Criando o gráfico

plt.figure(figsize=(8, 8))
plt.pie(cat_top5, labels=cat_top5.index)
plt.title('5 categorias com mais vendas')
plt.axis('equal')
plt.show()

In [ ]:
# Filtrando para obter somente valores da categoria 5
cat5 = vendas_futuras[vendas_futuras['Category'] == 5]

# Crieando barplot com Seaborn
sns.set()
sns.barplot(x='Month', y='predicted_sales', data=cat5)

# Adicionando titulo e legendas
plt.title('Previsões de vendas da categoria 5')
plt.xlabel('Meses')
plt.ylabel('Previsão de Vendas')

plt.show()